In [13]:
# Processing pipeline to turn generated data into a format that is easy to read into tensorflow: TFRecord files.
# These files allow for fast read and write to Tensorflow architectures, and are preferred using the new Dataset API. 
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import imageio
import numpy
import glob
from skimage.transform import resize
import sys

# The documentation for this code is somewhat spotty, and has been cobbled together from a variety of sources:
# https://medium.com/@WuStangDan/step-by-step-tensorflow-object-detection-api-tutorial-part-2-converting-dataset-to-tfrecord-47f24be9248d
# http://machinelearninguru.com/deep_learning/tensorflow/basics/tfrecord/tfrecord.html
# https://www.skcript.com/svr/why-every-tensorflow-developer-should-know-about-tfrecord/
# tensorflow/inception/inception/data/build_image_Data.py

# Once within the format of tfrecord files, it is easier to combine datasets, to split them, etc, so we just treat
# these on a video by video basis. 


In [14]:
## Designate helper function to define features for examples more easily
def _int64_feature_(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature_(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _float_feature_(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _full_example_(videoname,reader,i):
    resize_param = 128
    img_0 = reader.get_data(i)
    img_0_ds = resize(img_0,(resize_param,resize_param))
    

    features = {'video': _bytes_feature_(tf.compat.as_bytes((videoname))),
                'frame': _int64_feature_(i),
                'image': _bytes_feature_(tf.compat.as_bytes(img_0_ds.tostring())),
                }
    example = tf.train.Example(features = tf.train.Features(feature = features))
    return example

def _encoder_example_(videoname,reader,i):
    resize_param = 128
    img_0 = reader.get_data(i)
    img_0_ds = resize(img_0,(resize_param,resize_param))
    intensity = img_0_ds[int(resize_param/2),int(resize_param/2),0]
    print(intensity)
    features = {'video': _bytes_feature_(tf.compat.as_bytes((videoname))),
                'frame': _int64_feature_(i),
                'image': _bytes_feature_(tf.compat.as_bytes(img_0_ds.tostring())),
                'intensity':_float_feature_(intensity)
                }
    example = tf.train.Example(features = tf.train.Features(feature = features))
    return example

In [15]:
## To generate a tfrecords file for full training

def vids_to_fulltrain(datadirectory,extension):
    # We iterate through all directories
    videos = glob.glob(datadirectory)
    for videoname in videos:
        print('working on video: ' +videoname)
        # Choose a video, and load it
        video = imageio.get_reader(videoname,'ffmpeg')
        # We want to open a corresponding tfRecords file
        datafile_name = videoname.split('.mp4')[0]+extension
        writer = tf.python_io.TFRecordWriter(datafile_name)
        nb_frames = video.get_length()
        for i in range(nb_frames):
            # Iterate through frames
            example = _full_example_(videoname,video,i)
            writer.write(example.SerializeToString())
        writer.close()
        sys.stdout.flush()

datadirectory = 'datadirectory/toydynamics_nograv/*.mp4'
extension = 'train.tfrecords'
vids_to_fulltrain(datadirectory,extension)

working on video: datadirectory/toydynamics_nograv/Video_ball_color_corrback.mp4


/Users/taigaabe/anaconda3/envs/constrainedgen/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


working on video: datadirectory/toydynamics_nograv/Video_ball_color_corrback_noise.mp4
working on video: datadirectory/toydynamics_nograv/Video_ball_color.mp4
working on video: datadirectory/toydynamics_nograv/Video_ball_color_small.mp4
working on video: datadirectory/toydynamics_nograv/Video_ball_color_med.mp4
working on video: datadirectory/toydynamics_nograv/Video_ball_color_noback.mp4
working on video: datadirectory/toydynamics_nograv/Video_ball_color_big.mp4
working on video: datadirectory/toydynamics_nograv/Video_ball_color_dynback.mp4


In [12]:
## To generate a tfrecords file for encoder only (VIN) training
datadirectory = 'datadirectory/toydynamics_nograv/*.mp4'
extension = 'encoder_train.tfrecords'

def vids_to_encodertrain(datadirectory,extension): ## assumes the ball is in the center 
    # We iterate through all directories
    videos = glob.glob(datadirectory)
    for videoname in videos:
        print('working on video: ' +videoname)
        # Choose a video, and load it
        video = imageio.get_reader(videoname,'ffmpeg')
        # We want to open a corresponding tfRecords file
        datafile_name = videoname.split('.mp4')[0]+extension
        writer = tf.python_io.TFRecordWriter(datafile_name)
        nb_frames = video.get_length()
        for i in range(nb_frames):
            # Iterate through frames
            example = _encoder_example_(videoname,video,i)
            writer.write(example.SerializeToString())
        writer.close()
        sys.stdout.flush()

working on video: datadirectory/toydynamics_nograv/Video_ball_color_corrback.mp4
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887


/Users/taigaabe/anaconda3/envs/constrainedgen/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648743887
0.5123130648

0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633

0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633425262
0.4951705633

...

0.9372549019607843
0.9058823529411765
0.8588235294117645
0.807843137254902
0.7529411764705882
0.6941176470588235
0.6313725490196078
0.5725490196078431
0.5019607843137255
0.43529411764705883
0.3686274509803922
0.3058823529411765
0.24705882352941178
0.19215686274509808
0.1450980392156863
0.10196078431372549
0.06274509803921569
0.03529411764705882
0.011764705882352941
0.00392156862745098
0.0
0.00392156862745098
0.0196078431372549
0.0392156862745098
0.07450980392156863
0.10980392156862745
0.1568627450980392
0.2117647058823529
0.26666666666666666
0.3215686274509804
0.3882352941176471
0.4509803921568627
0.5176470588235295
0.5843137254901961
0.6470588235294118
0.7058823529411765
0.7686274509803923
0.8156862745098039
0.8705882352941177
0.9098039215686275
0.9450980392156862
0.9725490196078431
0.9882352941176471
1.0
1.0
0.9921568627450981
0.9764705882352942
0.9490196078431372
0.9176470588235295
0.8784313725490196
0.8313725490196079
0.7764705882352942
0.7215686274509804
0.6588235294117647
0.6
0.5

0.2117647058823529
0.1568627450980392
0.11372549019607844
0.07450980392156863
0.0392156862745098
0.0196078431372549
0.00784313725490196
0.0
0.00392156862745098
0.011764705882352941
0.03529411764705882
0.06274509803921569
0.09411764705882353
0.1411764705882353
0.19215686274509808
0.24705882352941178
0.3058823529411765
0.36470588235294116
0.4274509803921569
0.4980392156862745
0.5647058823529412
0.6274509803921569
0.6901960784313725
0.7490196078431373
0.803921568627451
0.8549019607843138
0.8980392156862744
0.9333333333333333
0.9647058823529413
0.9882352941176471
0.996078431372549
1.0
0.996078431372549
0.9803921568627451
0.9607843137254903
0.9333333333333333
0.8901960784313725
0.8509803921568626
0.796078431372549
0.7333333333333333
0.6784313725490196
0.615686274509804
0.5490196078431373
0.48235294117647065
0.4196078431372549
0.35294117647058826
0.29411764705882354
0.2392156862745098
0.1843137254901961
0.13333333333333333
0.09019607843137255
0.054901960784313725
0.03137254901960784
0.011764

working on video: datadirectory/toydynamics_nograv/Video_ball_color_dynback.mp4
0.9921568627450981
0.9921568627450981
0.9921568627450981
0.9607843137254903
0.9254901960784314
0.9058823529411765
0.8666666666666667
0.8
0.7490196078431373
0.6901960784313724
0.6374560786228554
0.5611606373506433
0.48235294117647065
0.43094566195618883
0.3834844851026348
0.3067881864659927
0.24884446088005527
0.19135365205652588
0.13984374999999996
0.11505821078431377
0.08235294117647059
0.0392156862745098
0.017018995098039245
0.00784313725490196
0.00784313725490196
0.022196691176470558
0.0196078431372549
0.027450980392156862
0.058823529411764705
0.0784313725490196
0.1607843137254902
0.20696333342907475
0.26659007352941166
0.3315737256816789
0.392156862745098
0.4666666666666667
0.5254901960784314
0.588082107843137
0.6431372549019608
0.7058823529411765
0.803921568627451
0.8313725490196079
0.8549019607843138
0.9425091911764707
0.9699601715686275
0.9803921568627451
1.0
1.0
0.984313725490196
0.9812719606885723


0.2
0.1450980392156863
0.10196078431372549
0.058823529411764705
0.03529411764705882
0.010935465494791775
0.0
0.0
0.0
0.0
0.054901960784313725
0.07450980392156863
0.12156862745098039
0.17647058823529413
0.20517769607843134
0.28619791666666655
0.324234068627451
0.3908241421568627
0.4766717648973651
0.5371783088235293
0.5995731129365808
0.6653339460784313
0.7289588479434742
0.784313725490196
0.8745098039215686
0.8823529411764706
0.9137254901960783
0.9607843137254903
0.9712928921568628
1.0
0.9773503920611214
0.9712928921568628
0.9712928921568628
0.9687040441176472
0.9468576468673406
0.9171941420611214
0.8539455638212314
0.8191549263748468
0.7672947303921569
0.6902726715686276
0.615686274509804
0.5647058823529412
0.512469362745098
0.45349264705882336
0.38861161774280006
0.324157475490196
0.2531163234336703
0.20909926470588225
0.1612372304878983
0.10633526970358456
0.08235294117647059
0.03137254901960784
0.0392156862745098
0.00784313725490196
0.00392156862745098
0.00784313725490196
0.0126445

0.4235294117647058
0.388235294117647
0.29411764705882354
0.25138274548100487
0.2100556616689645
0.14901960784313725
0.09856872558593766
0.0784313725490196
0.0392156862745098
0.011764705882352941
0.00784313725490196
0.019684436274509887
0.01568627450980392
0.012217622644761058
0.0196078431372549
0.054901960784313725
0.10196078431372549
0.1411764705882353
0.21168811274509788
0.2536458333333334
0.3437653186274509
0.3764705882352941
0.46309539196537974
0.52410691205193
0.5738817401960784
0.6326286764705882
0.6941176470588235
0.7686274509803923
0.803921568627451
0.8784313725490196
0.9019607843137254
0.9333333333333333
0.9647058823529413
1.0
1.0
1.0
0.9817248774509804
0.9882352941176471
0.9764705882352942
0.9498228185317092
0.8862745098039215
0.8392156862745098
0.7711657355813417
0.7232776716643688
0.6444699754901961
0.6326547061695771
0.5368019851983762
0.472220686370251
0.41443014705882364
0.3451240689146753
0.2784313725490196
0.21910436293658078
0.1607843137254902
0.13725490196078433
0.07